In [165]:
from subprocess import Popen, PIPE
from pathlib import Path
from tempfile import NamedTemporaryFile
import re
import itertools
from clang.cindex import Index, CursorKind, SourceLocation, Cursor

In [2]:
def grep_cpp_files(*args):
    if not args: args = ['SP_SETATTR', 'src/']
    proc = Popen(['grep', '-rn', *args], stdout=PIPE)
    res = (line.strip().decode().split(':',2) for line in proc.stdout)
    res = ((fn, [(int(line), source) for _, line, source in g])
           for fn, g in itertools.groupby(res, lambda x: x[0]))
    res = list(res)
    assert proc.wait() == 0
    return res

f, occs = grep_cpp_files()[0]
line, source = occs[0]
lines = [l.strip() for l in open(f)]

In [45]:
def index_includes(includes):
    tmp = NamedTemporaryFile('w', suffix='.cpp')
    tmp.write('\n'.join(['#include "%s"' % str(Path(f).absolute()) for f in includes]))
    tmp.file.flush()

    index = Index.create()
    args='-std=c++17 -I./src -I/usr/lib/clang/5.0.1/include/ -I../../pyGATB-build/ext/gatb-core/include/Release/'.split()
    tu = index.parse(tmp.name, args=args)

    tmp.close()
    return tu

tu = index_includes([f for f, occs in grep_cpp_files()])
for d in tu.diagnostics:
    if d.severity > 3:
        print(d.spelling, d.location)

In [181]:
def bfs(c, matcher):
    children = c.get_children()
    while children: 
        next_level = []
        for child in children:
            if matcher(child): return child
            else: next_level.extend(child.get_children())
        children = next_level
        
        
r = bfs(c, matcher=lambda c: c.kind == CursorKind.MEMBER_REF_EXPR)

In [297]:
class SourceManager:
    def __init__(self):
        self.files = {}
        
    def __getitem__(self, fname):
        fname = Path(fname).resolve()
        assert fname.is_file()
        file = self.files.get(fname)
        if file is not None:
            return file
        else:
            file = Source(fname)
            self.files[fname] = file
            return file
        
    def modified(self):
        for file in self.files.values():
            if file.modified:
                yield file
                
    def save(self):
        for file in self.files.values():
            file.save()
        
class Source:
    def __init__(self, fname):
        self.fname = fname
        self.lines = open(fname, 'rt').read().split('\n')
        self.modified = False
    def __repr__(self):
        return '<File "%s"%s>' % (self.fname.relative_to(Path.cwd()), " modified" if self.modified else "")
    
    def __getitem__(self, line):
        assert line > 0
        return self.lines[line-1]
    
    def __setitem__(self, line, source):
        self.modified = self.lines[line-1] != source
        self.lines[line-1] = source
        
    def save(self):
        if not self.modified: return False
        open(self.fname, 'w').write('\n'.join(self.lines))
        return True
        
    
sm = SourceManager()

In [301]:
spsetattr_re = re.compile(r'SP_SETATTR\s*\((\w+?)\)\s*;')
sm = SourceManager()

for f, occs in grep_cpp_files():
    lines = [l for l in open(f)]
    for line, source in occs:
        m = spsetattr_re.search(source)
        if not m:
            print("Regexp failled:", source)
            continue
        
        setter_file = sm[f]
        assert setter_file[line] == source
        #setter_file[line] = spsetattr_re.sub(r'_\1 = \1;', setter_file[line])
        #assert setter_file[line] != source
            
        c = Cursor.from_location(tu, tu.get_location(f, (line, m.start()+1)))
        c = bfs(c, matcher=lambda c: c.kind == CursorKind.MEMBER_REF_EXPR)
        if not c or not c.spelling.startswith('_'):
            print("Could not find attribute declaration :", source)
            continue

        defloc = c.referenced.location   
        deffile = sm[defloc.file.name]
        defline = deffile[defloc.line]
        
        deffile[defloc.line] = re.sub(r'(\w.*?)\s*\*', r'std::shared_ptr<\1>', defline, 1)

        #print(defline)


Regexp failled:     void setNodeCache   (NodeCacheMap*          nodecache)  { _nodecache = nodecache; /* would like to do "SP_SETATTR (nodecache)" but nodecache is an unordered_map, not some type that derives from a smartpointer. so one day, address this. I'm not sure if it's important though. Anyway I'm phasing out NodeCache in favor of GraphUnitigs. */; }
Regexp failled:  *  and destructors. Moreover, one can use the SP_SETATTR macro which eases this process.
Regexp failled:  *  \see SP_SETATTR
Regexp failled:  *  \see SP_SETATTR


In [302]:
sm.save()

In [269]:
Path.relative_to(Path('src').absolute(), 

PosixPath('src')

In [86]:
lines[line-1]

'void setRoot (Group* root)  { SP_SETATTR(root); }'

CursorKind.CXX_METHOD


In [ ]:
def 

In [144]:
for c in tu.cursor.from_location(tu,l).walk_preorder():
    print(c.kind)

CursorKind.CXX_METHOD
CursorKind.PARM_DECL
CursorKind.TYPE_REF
CursorKind.COMPOUND_STMT
CursorKind.COMPOUND_STMT
CursorKind.IF_STMT
CursorKind.BINARY_OPERATOR
CursorKind.UNEXPOSED_EXPR
CursorKind.MEMBER_REF_EXPR
CursorKind.UNEXPOSED_EXPR
CursorKind.DECL_REF_EXPR
CursorKind.COMPOUND_STMT
CursorKind.RETURN_STMT
CursorKind.IF_STMT
CursorKind.BINARY_OPERATOR
CursorKind.UNEXPOSED_EXPR
CursorKind.MEMBER_REF_EXPR
CursorKind.UNEXPOSED_EXPR
CursorKind.INTEGER_LITERAL
CursorKind.COMPOUND_STMT
CursorKind.CALL_EXPR
CursorKind.MEMBER_REF_EXPR
CursorKind.UNEXPOSED_EXPR
CursorKind.UNEXPOSED_EXPR
CursorKind.MEMBER_REF_EXPR
CursorKind.BINARY_OPERATOR
CursorKind.MEMBER_REF_EXPR
CursorKind.UNEXPOSED_EXPR
CursorKind.DECL_REF_EXPR
CursorKind.IF_STMT
CursorKind.BINARY_OPERATOR
CursorKind.UNEXPOSED_EXPR
CursorKind.MEMBER_REF_EXPR
CursorKind.UNEXPOSED_EXPR
CursorKind.INTEGER_LITERAL
CursorKind.COMPOUND_STMT
CursorKind.CALL_EXPR
CursorKind.MEMBER_REF_EXPR
CursorKind.UNEXPOSED_EXPR
CursorKind.UNEXPOSED_EXPR
Cur

In [245]:
lines[line-1]

'void setBufferData (tools::misc::Data* bufferData)  { SP_SETATTR(bufferData); }'

In [236]:
get_cpp_files()

[('src/gatb/bank/impl/BankBinary.hpp',
  161,
  '        void setBufferData (tools::misc::Data* bufferData)  { SP_SETATTR(bufferData); }'),
 ('src/gatb/bank/impl/BankConverterAlgorithm.hpp',
  83,
  '    void setBankInput (IBank* bankInput)  { SP_SETATTR(bankInput); }'),
 ('src/gatb/bank/impl/BankConverterAlgorithm.hpp',
  86,
  '    void setBankOutput (IBank* bankOutput)  { SP_SETATTR(bankOutput); }'),
 ('src/gatb/bank/impl/BankHelpers.hpp',
  125,
  '    void setRef (IBank* ref)  { SP_SETATTR(ref); }'),
 ('src/gatb/bank/impl/BankRandom.hpp',
  116,
  '        void setDataRef (tools::misc::Data* dataRef)  { SP_SETATTR(dataRef); }'),
 ('src/gatb/bank/impl/BankSplitter.hpp',
  130,
  '        void setDataRef (tools::misc::Data* dataRef)  { SP_SETATTR(dataRef); }'),
 ('src/gatb/bank/impl/BankSplitter.hpp',
  133,
  '        void setItRef (tools::dp::Iterator<Sequence>* itRef)  { SP_SETATTR(itRef); }'),
 ('src/gatb/bank/impl/BankSplitter.hpp',
  149,
  '    void setReference (IBank* refer

In [237]:
for k, g in itertools.groupby(get_cpp_files(), key=lambda x: x[0]):
    print(k, list(g))

src/gatb/bank/impl/BankBinary.hpp [('src/gatb/bank/impl/BankBinary.hpp', 161, '        void setBufferData (tools::misc::Data* bufferData)  { SP_SETATTR(bufferData); }')]
src/gatb/bank/impl/BankConverterAlgorithm.hpp [('src/gatb/bank/impl/BankConverterAlgorithm.hpp', 83, '    void setBankInput (IBank* bankInput)  { SP_SETATTR(bankInput); }'), ('src/gatb/bank/impl/BankConverterAlgorithm.hpp', 86, '    void setBankOutput (IBank* bankOutput)  { SP_SETATTR(bankOutput); }')]
src/gatb/bank/impl/BankHelpers.hpp [('src/gatb/bank/impl/BankHelpers.hpp', 125, '    void setRef (IBank* ref)  { SP_SETATTR(ref); }')]
src/gatb/bank/impl/BankRandom.hpp [('src/gatb/bank/impl/BankRandom.hpp', 116, '        void setDataRef (tools::misc::Data* dataRef)  { SP_SETATTR(dataRef); }')]
src/gatb/bank/impl/BankSplitter.hpp [('src/gatb/bank/impl/BankSplitter.hpp', 130, '        void setDataRef (tools::misc::Data* dataRef)  { SP_SETATTR(dataRef); }'), ('src/gatb/bank/impl/BankSplitter.hpp', 133, '        void setItR

In [235]:
list(_)

[]

In [220]:
re.sub(r'SP_SETATTR\((\w+?)\);', r'_\1 = \1;', source)

'        void setBufferData (tools::misc::Data* bufferData)  { _bufferData = bufferData; }'

In [70]:

index = Index.create()

In [56]:
def print_children(c, select_name=None, select_kind=None):
    res = []
    for child in c.get_children():
        print(child.kind.name, child.spelling)
        if child.spelling == select_name or child.kind == select_kind:
            res.append(child)
    if res: return res

In [45]:
args='-I./src -I/usr/lib/clang/5.0.1/include/'.split()
tu = index.parse(f, args=args)
for d in tu.diagnostics:
    if d.severity > 3:
        print(d.spelling)

In [60]:
def get_gatb_namespace(tu):
    lastgatb = None
    for child in tu.cursor.get_children():
        if child.kind == CursorKind.NAMESPACE and child.spelling == 'gatb':
            lastgatb = child
    return lastgatb

gatb = get_gatb_namespace(tu)

In [124]:
def find_classes(c):
    for child in c.get_children():
        if child.kind == CursorKind.NAMESPACE:
            yield from find_classes(child)
        elif child.kind == CursorKind.CLASS_DECL:
            yield child
            yield from find_classes(child)
            
c = list(find_classes(gatb))[1]

In [125]:
c = print_children(c, 'setBufferData')[0]

CXX_BASE_SPECIFIER tools::dp::Iterator<Sequence>
CXX_ACCESS_SPEC_DECL 
CONSTRUCTOR Iterator
DESTRUCTOR ~Iterator
CXX_METHOD first
CXX_METHOD next
CXX_METHOD isDone
CXX_METHOD item
CXX_METHOD estimate
CXX_ACCESS_SPEC_DECL 
FIELD_DECL _ref
FIELD_DECL _isDone
FIELD_DECL _bufferData
CXX_METHOD setBufferData
FIELD_DECL cpt_buffer
FIELD_DECL blocksize_toread
FIELD_DECL nseq_lues
FIELD_DECL binary_read_file
FIELD_DECL _index


In [129]:
stmt = print_children(c, select_kind=CursorKind.COMPOUND_STMT)[0]
stmt = print_children(stmt, select_kind=CursorKind.COMPOUND_STMT)[0]

PARM_DECL bufferData
COMPOUND_STMT 
COMPOUND_STMT 
NULL_STMT 


In [137]:
c = stmt

In [160]:
extent = c.extent
start = extent.start
sline = start.line
scol = start.column
end = extent.end
eline = end.line
ecol = end.column
fname = c.extent.end.file.name

inrange = False
out = []
for i,line in enumerate(open(fname, 'rb'), 1):
    if i == sline: inrange = True
        
    if inrange:
        print(line)
        if i == eline:
            inrange = False
            if i == sline:
                out.append(line.strip()[scol+1:ecol+1])
            else:
                out.append(line[:ecol])
        else:
            if i == sline:
                out.append(line.strip()[scol:])
            out.append(line.strip())
    
out, 

b'        void setBufferData (tools::misc::Data* bufferData)  { SP_SETATTR(bufferData); }\n'


([b'(bufferData); }'],)

In [162]:
f = tu.get_file(f)

In [175]:
stmt = tu.cursor.from_location(tu, extent.start)
stmt = print_children(stmt, select_kind=CursorKind.BINARY_OPERATOR)[0]
stmt = print_children(stmt, select_kind=CursorKind.MEMBER_REF_EXPR)[0]

IF_STMT 
IF_STMT 
BINARY_OPERATOR 
IF_STMT 
MEMBER_REF_EXPR _bufferData
UNEXPOSED_EXPR bufferData


In [179]:
stmt.spelling

'_bufferData'

In [171]:
stmt=stmt[0[0]

MEMBER_REF_EXPR _bufferData
UNEXPOSED_EXPR bufferData


In [105]:
c = list(child.get_arguments())[0]
t = c.type.get_canonical()


TypeKind.POINTER